This notebook has been **adopted** from https://github.com/numba/numba-examples/tree/master/tutorials/nasa_apps_oct_2019: **Many thanks** to the original authors!

# An Introduction to Numba



## Why does Numba exist?
The idea surrounding Numba is to create the ability to extend Python with Python itself. It gets your code performance/capabilities that you just cannot otherwise achieve without C extensions/Cython/Pythran/&lt;insert extension creation package that calls a compiler here&gt;!

## Conventions used throughout this tutorial series:

 * Tasks (exercises for the reader) are marked up in bold blue, like this:<h3><span style="color:blue"> Task 1 </span></h3><br>

 * Hints/tips/suggestions/things to think about are in bold green, like this:
<h4><span style="color:green"> HINT: this is a hint </span></h4>


## What is Numba?!

**Numba is a type specialising just-in-time (JIT) compiler for Python functions.**

It is most often used via a decorator...

Simplest example with the simplest decorator:

In [3]:
from numba import njit

@njit
def add_one(x):
    return x + 1

In the above the ``njit`` decorator was imported and applied to a simple function ("Numba is ... for Python functions"). No compilation has occurred yet! Let's call the function with an integer:

<h3><span style="color:blue"> Task 1: call the `add_one` function with an integer input</span></h3>

In [ ]:
# You do this! Call `add_one` with an integer here...

🎉 Congratulations 🎉, you just used one of the world's most powerful compliation chains (LLVM!)!

Invoking the function makes compilation take place, this is what the just-in-time part is about ("Numba is a ... just-in-time compiler"). No compilation occurs until it is needed, and once it has occurred it is cached in process to be reused. Calling the function again with another integer will use the cached code path, it does not need to recompile.

In [6]:
add_one(-129)

-128

Now let's call function with a float.

In [7]:
add_one(12.34)

13.34

This of course works, but a new specialisation was needed, Numba recompiled the function for floats.

<h4><span style="color:green">Remember: Numba is a compiler, specialisation leads to optimisation</span></h4>


Each time a new **TYPE** of input is encountered a new specialisation for the function is being compiled. We can look at these via the `.signatures` attribute:

In [8]:
add_one.signatures

[(int64,), (float64,)]

this is what the type specilisation part means (Numba is a type specialising ... compiler).

As the `.signatures` attribute is simply a list of signatures (each new signature is just appended to the list) it's easy to retrieve a particular function signature. For example, this selects the signature for the first specialisation (the one with the integer argument), it's at index zero as this type was encountered first:

In [9]:
add_one.signatures[0]

(int64,)

It in fact has to work out the type of every variable based on the input types and the operations in the function, this is called "typing" a function and happens through a process called "type inference".

## How does Numba work?

![How Numba Works](How_Numba_Works.png)

#### <span style="color:green"> TIP: Understanding type inference is really important, most errors you are going to see are because Numba couldn't figure out what type something should be. <br><br>Numba takes a function in a dynamic language and makes a type-static compiled verion of it!</span> 

## What did Numba do?!

We've seen that Numba does multiple stages of transforms to go from bytecode to machine code. Three places that are important to look at are:
1. The Numba intermediate representation (IR) level, this shows type information.
   Access it via `.inspect_types` on a Numba JIT decorated function.
2. The LLVM IR level, this shows the LLVM IR that Numba generated to pass to LLVM.
   Access it via `.inspect_llvm` on a Numba JIT decorated function.
3. The assembly level, this shows the disassembly of the compiled object.
   Access it via `.inspect_asm` on a Numba JIT decorated function.

Each of these methods takes a signature `kwarg` (like one obtained from `.signatures` above) as these representations are also type-specialised.

<h3><span style="color:blue"> Task 2: Using the `add_one` function from above, for the `float64` signature take a look at the various levels of transforms.</span></h3>

In [ ]:
# first select the `float64` signature
my_sig = # You write this!
print(my_sig)

# Level 1, print the types using `.inspect_types()`, make sure to specify the `signature` kwarg
print(add_one.inspect_types())

In [ ]:
# Level 2, print the LLVM IR using `.inspect_llvm()`
print(# You write this!)

In [ ]:
# Level 3, print the disassembly using `.inspect_asm()`
print(# You write this!)

# Benchmarking

As we saw above, Numba JIT compiles specialisation of a function based on the input types. This means the first invocation is going to involve compilation, and subsequent are going to be from cache. Let's define a more involved function and look at how to benchmark Numba:

In [ ]:
import math

@njit
def hypot(x, y):
    # Implementation from https://en.wikipedia.org/wiki/Hypot
    x = abs(x);
    y = abs(y);
    t = min(x, y);
    x = max(x, y);
    t = t / x;
    return x * math.sqrt(1+t*t)

The `%timeit` magic is appropriate for timing Numba, it runs repeated loops so the cost of compiling the function is amortized.

In [ ]:
%timeit hypot(3.0, 4.0)

All Numba `@njit` decorated functions have a `.py_func` attribute which holds reference to the pure Python function, we can use this to check the Python interpreter performance for the above function.

In [ ]:
%timeit hypot.py_func(3.0, 4.0)

#### <span style="color:green"> **TIP: The most common mistake make when reporting performance results is reporting the execution time as the compilation time along with the execution time.** </span>

## Loops?!

NumPy is amazing, but a side effect of the "vectorisation" paradigm is having to avoid loops. Numba is a compiler, compilers like loops.

In [ ]:
import numpy as np
N = 1000
np.random.seed(0)
X = np.random.rand((N * N)).reshape((N,  N))

@njit
def awkward_sine(a):
    return np.imag(np.exp(1j * a))

# check result
np.testing.assert_allclose(awkward_sine(X), np.sin(X))

Check the performance of the NumPy implementation running in the interpreter:

In [ ]:
%timeit awkward_sine.py_func(X)

Check the performance of Numba JIT version:

In [ ]:
%timeit awkward_sine(X)

<h3><span style="color:blue"> Task 3a: Reimplement the above `awkward_sine` function with explicit looping using the template below...</span></h3>

In [ ]:
@njit
def awkward_sine_loops(a):
    m, n = a.shape
    result = np.empty_like(a)
    # You write this part!
    return result

# check result
np.testing.assert_allclose(awkward_sine_loops(X), np.sin(X))

<h3><span style="color:blue"> Task 3b: Check the performance of the loop based function above: </span>

In [ ]:
%timeit awkward_sine_loops(X)

## Assessing Performance

Above we saw Numba JIT out perform NumPy in both a vectorised and a loop induced version. The loop version was quickest by some margin.

#### <span style="color:green"> Discussion: What might happen in this similar case? </span>

In [ ]:
@njit
def awkward_exp(a):
    return np.cosh(a) + np.sinh(a)

# check result
np.testing.assert_allclose(awkward_exp(X), np.exp(X))

Check the performance of the NumPy implementation running in the interpreter:

In [ ]:
%timeit awkward_exp.py_func(X)

Check the performance of Numba JIT version:

In [ ]:
%timeit awkward_exp(X)

Above we saw that loops made a difference, try that trick again...

In [ ]:
@njit
def awkward_exp_loops(a):
    m, n = a.shape
    result = np.empty_like(a)
    for i in range(m):
        for j in range(n):
            result[i, j] = np.cosh(a[i, j]) + np.sinh(a[i, j])
    return result
            
# time it
%timeit awkward_exp_loops(X)

Let's switch on `fastmath`, this plumbs in Intel's Short Vector Math library (SVML) which swaps out `libm` functions for vectorized ones.

In [ ]:
@njit(fastmath=True)
def awkward_exp_loops_fastmath(a):
    m, n = a.shape
    result = np.empty_like(a)
    for i in range(m):
        for j in range(n):
            result[i, j] = np.cosh(a[i, j]) + np.sinh(a[i, j])
    return result

           
%timeit awkward_exp_loops_fastmath(X)

That made things a bit quicker but NumPy beat Numba all both cases...

#### <span style="color:green"> Answer: Anaconda NumPy ``ufunc`` behaviour is hugely optimised and uses MKL VML internally. </span>


## Performance Tips:

Do:
 * Write good tests for the correctness of your code before using the JIT
 * Write a good performance harness for your code:
   * Use real data/representative input
   * Keep track of measurements

Do not:

* Include compilation time in the execution time.
* Drag race tiny functions as an assessment of Numba's ability, Numba has dispatch cost:

In [ ]:
%timeit add_one(1)
%timeit add_one.py_func(1)

* Drag race fundamental math operations:

In [ ]:
@njit
def jit_cos(x):
    return math.cos(x)

%timeit jit_cos(12.34)
%timeit jit_cos.py_func(12.34)

* Expect Numba to outperform BLAS:

In [ ]:
@njit
def dgemv(A, x):
    m, n = A.shape
    acc = np.zeros((m,))
    for i in range(m):
        for j in range(n):
            acc[i] += A[i, j] * x[j]
    
@njit
def call_dot(A, x):
    return np.dot(A, x) # Numba will defer this to BLAS::XGEMV

%timeit dgemv(X, X[0])
%timeit call_dot(X, X[0])
%timeit call_dot.py_func(X, X[0])

You wouldn't expect Fortran/C to do any better in the above situations, Numba is the same, internally it tries to make sensible decisions.

## The importance of measurement:

Find the root of `f(x) = cos(x) + 1`. Use the Newton-Raphson algorithm. Use SciPy `optimize.newton` as a performance baseline.

In [ ]:
from scipy import optimize

def f(x):
    return np.cos(x) + 1

def dfdx(x):
    return -np.sin(x)

tol = 1e-7 # convergence tolerance
max_it = 50 # maximum iterations
x0 = 0.5 # starting guess

%timeit optimize.newton(f, x0, fprime=dfdx, tol=tol, maxiter=max_it)

<h3><span style="color:blue"> Task 4:  Write your own Newton-Raphson implementation based on based on pseudo-code from <a href="https://en.wikipedia.org/wiki/Newton%27s_method#Pseudocode">Wikipedia</a>.</span></h3>

In [ ]:
@njit
def NR_root(f, dfdx, x0, max_it=max_it, eps=tol):
    converged = False
    # You write this part!
    if converged:
        return x1
    else:
        raise RuntimeError("Solution did not converge")

# if the above is "correct", this will pass
np.testing.assert_allclose(NR_root.py_func(f, dfdx, x0), np.pi)

Time the Python function:

In [ ]:
%timeit NR_root.py_func(f, dfdx, x0)

Time the Python function with JIT compiled objective functions:

In [ ]:
f_jit = njit(f)
dfdx_jit = njit(dfdx)

%timeit NR_root.py_func(f_jit, dfdx_jit, x0)

Time the JIT compiled function with JIT compiled objective functions:

In [ ]:
%timeit NR_root(f_jit, dfdx_jit, x0)

Time SciPy with JIT compiled objective functions:

In [ ]:
%timeit optimize.newton(f_jit, x0, fprime=dfdx_jit, tol=tol, maxiter=max_it)

Let's really lean on the fact we have a specialising compiler... performance comes from specialisation.

In [ ]:
def specialize(f, dfdx):
    # Close over the objective functions so they are "constant"
    # Numba will spot this and LLVM will inline them into the machine code
    @njit
    def NR_root(x0, max_it=max_it, eps=tol):
        # Copy your NR function body from above here
    return NR_root

f_NR_root = specialize(f_jit, dfdx_jit)

%timeit f_NR_root(x0, max_it, tol)

#### <span style="color:green"> Discuss: What happened in the above?!</span>
#### <span style="color:green"> TIP: Experiment and time things!</span>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=77a5caea-ff40-471d-8b4b-98dc66dd30c3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>